In [14]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
import math

In [15]:
# Load in the data
ca_demData = pd.read_csv("CA_CensusData.csv")
ca_suicData = pd.read_csv("ca_county_suicide_2015.csv")

In [16]:
# Rename column names for merge
ca_demData = ca_demData.rename(columns = {'County' : 'County_Names'})

# Merge data
all_data = pd.merge(ca_demData, ca_suicData, on='County_Names', how='outer')

In [17]:
# Renaming some columns for use
all_data = all_data.rename(columns = {'Population, Census, April 1, 2010' : 'Population_2010', 'Persons per household, 2015-2019' : 'Avg_Household_Size', 'Veterans, 2015-2019' : 'Vet_Population', 'High school graduate or higher, percent of persons age 25 years+, 2015-2019' : 'HS_Grad_Percent', 'Bachelor\'s degree or higher, percent of persons age 25 years+, 2015-2019' : 'College_Grad_Percent', 'With a disability, under age 65 years, percent, 2015-2019' : 'Disability_percent', 'Persons  without health insurance, under age 65 years, percent' : 'Lacking_Health_Insurance_Percent', 'In civilian labor force, total, percent of population age 16 years+, 2015-2019' : 'Employed_16+_Percent', 'Total retail sales, 2012 ($1,000)' : 'Retail_Sales_by1000', 'Median household income (in 2019 dollars), 2015-2019' : 'Household_Income', 'Persons in poverty, percent' : 'Poverty_Percentage', 'All firms, 2012' : 'Firms_Num', 'Population per square mile, 2010' : 'Population_Density'})

# Dropping Alpine County as no data for suicide count
all_data = all_data.drop([1])
all_data.reset_index(drop=True, inplace=True)
all_data.head()

,County_Names,"Population estimates, July 1, 2019, (V2019)","Population estimates base, April 1, 2010, (V2019)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2019, (V2019)",Population_2010,"Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent","Female persons, percent","White alone, percent",...,"Women-owned firms, 2012","Minority-owned firms, 2012","Nonminority-owned firms, 2012","Veteran-owned firms, 2012","Nonveteran-owned firms, 2012",Population_Density,"Land area in square miles, 2010",FIPS_County_Code,Suicide_Counts,County_of_occurrence
0,Alameda County,1671329.0,1510258.0,10.7,1510271.0,5.7,20.3,14.3,50.7,49.3,...,57305.0,75026.0,69300.0,10213.0,134032.0,2043.6,739.02,1,138.0,1.0
1,Amador County,39752.0,38091.0,4.4,38091.0,4.1,15.1,27.5,45.6,89.6,...,921.0,259.0,2453.0,258.0,2363.0,64.1,594.58,5,7.0,5.0
2,Butte County,219186.0,220005.0,-0.4,220000.0,5.5,20.3,18.4,50.5,85.7,...,5341.0,1858.0,13644.0,1766.0,13509.0,134.4,1636.46,7,39.0,7.0
3,Calaveras County,45905.0,45578.0,0.7,45578.0,4.5,17.1,28.0,50.2,91.0,...,1008.0,291.0,2983.0,392.0,2799.0,44.7,1020.01,9,20.0,9.0
4,Colusa County,21547.0,21407.0,0.7,21419.0,6.9,27.1,14.9,49.1,91.1,...,306.0,397.0,656.0,55.0,944.0,18.6,1150.73,11,2.0,11.0


In [18]:
# Adding suicide rate column
all_data['Suicide_Rate'] = pd.Series(dtype='float')
# Adding risk type column
all_data['Risk_Type'] = pd.Series(dtype='int')

for i in range(57):
    # Suicide rate per 100,000 people
    all_data['Suicide_Rate'][i] = all_data['Suicide_Counts'][i]/all_data['Population_2010'][i] * 100000
    
    # High risk county if suicide rate greater than the national average 13.4
    if (all_data['Suicide_Counts'][i]/all_data['Population_2010'][i] * 100000) > 13.4:
        # high risk = 1
        all_data['Risk_Type'][i] = 1
    else:
        # low risk = 0
        all_data['Risk_Type'][i] = 0
        
all_data.head()

<ipython-input-18-b730b44b29bc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['Suicide_Rate'][i] = all_data['Suicide_Counts'][i]/all_data['Population_2010'][i] * 100000
<ipython-input-18-b730b44b29bc>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['Risk_Type'][i] = 0
<ipython-input-18-b730b44b29bc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['Risk_Type'][i] = 1


,County_Names,"Population estimates, July 1, 2019, (V2019)","Population estimates base, April 1, 2010, (V2019)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2019, (V2019)",Population_2010,"Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent","Female persons, percent","White alone, percent",...,"Nonminority-owned firms, 2012","Veteran-owned firms, 2012","Nonveteran-owned firms, 2012",Population_Density,"Land area in square miles, 2010",FIPS_County_Code,Suicide_Counts,County_of_occurrence,Suicide_Rate,Risk_Type
0,Alameda County,1671329.0,1510258.0,10.7,1510271.0,5.7,20.3,14.3,50.7,49.3,...,69300.0,10213.0,134032.0,2043.6,739.02,1,138.0,1.0,9.137433,0.0
1,Amador County,39752.0,38091.0,4.4,38091.0,4.1,15.1,27.5,45.6,89.6,...,2453.0,258.0,2363.0,64.1,594.58,5,7.0,5.0,18.377044,1.0
2,Butte County,219186.0,220005.0,-0.4,220000.0,5.5,20.3,18.4,50.5,85.7,...,13644.0,1766.0,13509.0,134.4,1636.46,7,39.0,7.0,17.727273,1.0
3,Calaveras County,45905.0,45578.0,0.7,45578.0,4.5,17.1,28.0,50.2,91.0,...,2983.0,392.0,2799.0,44.7,1020.01,9,20.0,9.0,43.880820,1.0
4,Colusa County,21547.0,21407.0,0.7,21419.0,6.9,27.1,14.9,49.1,91.1,...,656.0,55.0,944.0,18.6,1150.73,11,2.0,11.0,9.337504,0.0


In [19]:
# Splitting data into training and testing sets
# Using the 15 counties with the highest suicide rate and the 15 counties with the lowest suicide rate
# for the training set
count = 0
all_data = all_data.sort_values(by = 'Suicide_Rate')
training_data = pd.concat([all_data[0:20], all_data[37:57]])
testing_data = all_data[20:37]

In [20]:
# Splitting data up
x_train = training_data[['Avg_Household_Size', 'HS_Grad_Percent', 'College_Grad_Percent', 'Disability_percent', 'Vet_Population', 'Employed_16+_Percent', 'Lacking_Health_Insurance_Percent', 'Household_Income', 'Poverty_Percentage', 'Population_Density']]
y_train = training_data['Risk_Type']
x_test = testing_data[['Avg_Household_Size', 'HS_Grad_Percent', 'College_Grad_Percent', 'Disability_percent', 'Vet_Population', 'Employed_16+_Percent', 'Lacking_Health_Insurance_Percent', 'Household_Income', 'Poverty_Percentage', 'Population_Density']]
y_test = testing_data['Risk_Type']

In [21]:
# Creating tensors for NN
x, x_t = torch.from_numpy(pd.DataFrame.to_numpy(x_train)), torch.from_numpy(pd.DataFrame.to_numpy(x_test))
y, y_t = torch.from_numpy(pd.DataFrame.to_numpy(y_train)), torch.from_numpy(pd.DataFrame.to_numpy(y_test))

In [22]:
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.lin1 = nn.Linear(10, 5)
        self.lin2 = nn.Linear(5, 1)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.sigmoid(x)
        return x

net = MyNetwork().double()

In [23]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the nn.Linear
# module which is members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(net.parameters(), lr=0.0000000001)

In [24]:
for epoch in range(2):
    
    for t in range(40):     
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = net(x[t])

        # Compute and print loss
        loss = nn.L1Loss()
        loss_val = loss(y_pred, y)
        # print every 10 mini-batches
        if t % 5 == 0:    
            print(t, loss_val.item())

        # Perform a backward pass, and update the weights.
        loss_val.backward()
        optimizer.step()

print("Finished Training")

0 0.5
5 0.5
10 0.5
15 0.5
20 0.5
25 0.5
30 0.5
35 0.5
0 0.5
5 0.5
10 0.5
15 0.5
20 0.5
25 0.5
30 0.5
35 0.5
Finished Training


C:\Users\chris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([40])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [25]:
# Save model
torch.save(net.state_dict(), 'CA_PNN')

In [26]:
# Load model for testing
net.load_state_dict(torch.load('CA_PNN'))
outputs = net(x_t)

correct_count = 0

# Compute % accuracy
for i in range(17):
    if int(outputs[i].item()) == int(y_t[i].item()):
        correct_count += 1

print("Score: " + str(round(correct_count / 17 * 100, 2)) + "%")

Score: 82.35%
